In [1]:
from bs4 import BeautifulSoup
import re
import requests as rq

def getHTMLtext(url):
    try:
        r=rq.get(url,headers={"user-agent":"chrome/10"},timeout=30)
        r.raise_for_status() # If status != 200, cause HTTPError
        r.encoding=r.apparent_encoding
        return r.text
    except:
        return "Error occurred"

In [2]:
def name_p_has_no_strong_child(tag):
    if tag.name=="p" and tag.has_attr('class') and not tag.contents[0].name=="strong" and not tag.has_attr('align'):
        return True

In [3]:
def formatingLists(soup_findall_lst):
    reslst=[]
    imlst=[]
    for i in soup_findall_lst:
        if not i.a == None:
            hpath = i.a.attrs['href']
        else:
            hpath = None
        if len(i) == None or len(i) == 1:
            fname=i.string
        else:
            for k in i.strings:
                imlst.append(k)
            imstr=".".join(imlst).replace(' ','').replace(chr(12288),'').replace('/','').replace('\\','')
            if imstr.endswith('.pdf'):
                fname=imstr
            else:
                fname=imstr+'.pdf'
            imlst=[]
        reslst.append([fname,hpath])
    return reslst

# consider more efficiently truncate the filename or have to find strings in hpath!

In [4]:
def findDownloadPath(lst):
    hp='http://bilab.bnu.edu.cn/'
    root='D:/haruk/Documents/PythonCodes/pdf_dl/'
    durl=hp+lst[1]
    dpath=root+lst[0]
    return [durl,dpath]

In [5]:
def getPDFdown(url,path):
    try:
        html=rq.get(url,headers={"user-agent":"chrome/10"},timeout=30)
        html.raise_for_status() # If status != 200, cause HTTPError
        with open(path,"wb") as f:
            for chunk in html.iter_content(chunk_size=8192):
                f.write(chunk)
            f.close()
    except:
        return "Error occurred"

In [6]:
def main():
    url="http://bilab.bnu.edu.cn/publication.html"
    html=getHTMLtext(url)
    
    soup=BeautifulSoup(html,"html.parser")
    
    sfa_lst=soup(name_p_has_no_strong_child)
    print("Please check for the length of Soup_find_all lists is now: {}".format(len(sfa_lst)))

    comparLsts=formatingLists(sfa_lst)
    
    # Initiating download:
    itrial=0
    for ls in comparLsts[:6]:
        try:
            [durl,dpath]=findDownloadPath(ls)
            getPDFdown(durl,dpath)
            itrial+=1
            print("PDF file No.{} download success!".format(itrial))
        except:
            print("Error occurred at Comparison Table No.:{}".format(itrial))
            continue

# For now, only download the first 5 files!
main()

# need to dubug: how to truncate a valid file name for downloading?! 
# maybe should check for valid within function: findDownloadPath  rather than formattingLists

Please check for the length of Soup_find_all lists is now: 104
Error occurred at Comparison Table No.:0
PDF file No.1 download success!
PDF file No.2 download success!
PDF file No.3 download success!
PDF file No.4 download success!
PDF file No.5 download success!
